# Part 2: Citation Crawler + Selector

**SNU AI Psychology Workshop - February 2026**

---

## Part 1과의 차이

| Part 1 (API 검색) | Part 2 (Crawler + Selector) |
|-------------------|-----------------------------|
| 키워드 → 논문 목록 | **시드 논문 선택** → 인용 네트워크 확장 |
| 단일 검색 | 다중 홉 탐색 (multi-hop) |
| 검색 결과만 | 관련 논문 필터링 (Selector) |

## 학습 목표
1. **paperscraper**로 다중 DB 검색 (arXiv, PubMed, bioRxiv, chemRxiv)
2. **시드 논문 선택**: 번호 또는 DOI 직접 입력
3. **Citation Crawler**: 시드에서 인용 네트워크 확장
4. **SPECTER2 Selector**: 관련성 높은 논문 필터링
5. **시드 비교**: 다른 시드로 결과가 어떻게 달라지는지 확인

---
## Setup

In [ ]:
# Cell 1: 환경 설정
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("Colab 환경에서 실행 중")
except ImportError:
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치
if IN_COLAB:
    print("패키지 설치 중...")
    %pip install paperscraper requests pandas torch transformers adapters -q
    print("설치 완료!")
else:
    print("로컬 환경: 필요 시 아래 명령 실행")
    print("  pip install paperscraper requests pandas torch transformers adapters")

In [ ]:
# Cell 3: API Key 로딩 (선택사항)
import os

SEMANTIC_SCHOLAR_API_KEY = None

if IN_COLAB:
    try:
        from google.colab import userdata
        SEMANTIC_SCHOLAR_API_KEY = userdata.get('SEMANTIC_SCHOLAR_API_KEY', None)
    except:
        pass
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        SEMANTIC_SCHOLAR_API_KEY = os.getenv('SEMANTIC_SCHOLAR_API_KEY')
    except:
        pass

if SEMANTIC_SCHOLAR_API_KEY:
    print("✓ Semantic Scholar API Key 로드 성공!")
    print(f"  Key: {SEMANTIC_SCHOLAR_API_KEY[:10]}...")
    print(f"  허용량: 5000 req/5분")
else:
    print("✗ Semantic Scholar API Key 없음 (선택사항)")
    print("  허용량: 100 req/5분 (무료)")
    print("  발급 권장: https://www.semanticscholar.org/product/api")

In [ ]:
# Cell 4: 공통 import
import requests
import time
import pandas as pd
from typing import List, Dict, Set, Optional
from collections import defaultdict, Counter

print("공통 모듈 로드 완료")

---
## 1. paperscraper로 후보 논문 검색

[paperscraper](https://github.com/jannisborn/paperscraper)는 여러 학술 DB를 통합 검색하는 오픈소스입니다.

| 지원 DB | 특징 |
|---------|------|
| arXiv | CS, Physics 프리프린트 |
| PubMed | 의학, 심리학 |
| bioRxiv | 생물학 프리프린트 |
| chemRxiv | 화학 프리프린트 |

### 검색 쿼리 작성법

| 형식 | 예시 | 의미 |
|------|------|------|
| 단순 키워드 | `depression detection` | 두 단어 모두 포함 |
| AND 연산 | `depression AND anxiety` | 둘 다 포함 (명시적) |
| OR 연산 | `depression OR anxiety` | 둘 중 하나 |
| 복합 조건 | `(depression OR anxiety) AND NLP` | 그룹화 |
| 구문 검색 | `"major depressive disorder"` | 정확한 구문 |

**팁**: comma(`,`)보다 `AND`/`OR` 키워드가 더 명확합니다.

In [ ]:
# Cell 5: paperscraper 검색 함수

def search_with_paperscraper(query: str, max_results: int = 5) -> pd.DataFrame:
    """
    paperscraper로 여러 DB 검색
    
    Returns:
        DataFrame with columns: source, title, authors, year, doi, abstract
    """
    all_papers = []
    
    # arXiv 검색
    try:
        from paperscraper.arxiv import get_arxiv_papers_api
        print("[arXiv] 검색 중...")
        arxiv_df = get_arxiv_papers_api(query, max_results=max_results, verbose=False)
        for _, row in arxiv_df.iterrows():
            all_papers.append({
                'source': 'arXiv',
                'title': row.get('title', ''),
                'authors': row.get('authors', []),
                'year': str(row.get('date', ''))[:4] if row.get('date') else '',
                'doi': row.get('doi', ''),
                'abstract': str(row.get('abstract', ''))[:200] + '...' if row.get('abstract') else ''
            })
        print(f"  -> {len(arxiv_df)}개 발견")
    except Exception as e:
        print(f"[arXiv] 오류: {e}")
    
    # PubMed 검색
    try:
        from paperscraper.pubmed import get_pubmed_papers
        print("[PubMed] 검색 중...")
        pubmed_df = get_pubmed_papers(query, max_results=max_results)
        for _, row in pubmed_df.iterrows():
            all_papers.append({
                'source': 'PubMed',
                'title': row.get('title', ''),
                'authors': row.get('authors', []),
                'year': str(row.get('date', ''))[:4] if row.get('date') else '',
                'doi': row.get('doi', ''),
                'abstract': str(row.get('abstract', ''))[:200] + '...' if row.get('abstract') else ''
            })
        print(f"  -> {len(pubmed_df)}개 발견")
    except Exception as e:
        print(f"[PubMed] 오류: {e}")
    
    # bioRxiv 검색 (선택사항)
    try:
        from paperscraper.get_dumps import biorxiv
        print("[bioRxiv] 검색 중... (시간 소요)")
        # bioRxiv는 dump 기반이라 시간이 오래 걸릴 수 있음
        # 워크샵에서는 스킵 가능
        print("  -> 스킵 (시간 절약)")
    except Exception as e:
        print(f"[bioRxiv] 스킵")
    
    return pd.DataFrame(all_papers)

print("search_with_paperscraper() 정의 완료")

In [ ]:
# Cell 6: 후보 논문 검색 실행
QUERY = "depression detection natural language processing"  # <- 수정 가능!

print(f"검색 쿼리: {QUERY}")
print("=" * 50)

candidates_df = search_with_paperscraper(QUERY, max_results=5)

print(f"\n총 {len(candidates_df)}개 후보 논문 발견")

In [ ]:
# Cell 7: 후보 논문 테이블로 보기
if len(candidates_df) > 0:
    # 번호 추가
    candidates_df.insert(0, 'No.', range(1, len(candidates_df) + 1))
    
    # 제목 축약
    candidates_df['title_short'] = candidates_df['title'].apply(
        lambda x: x[:50] + '...' if len(str(x)) > 50 else x
    )
    
    # 표시용 컬럼 선택
    display_cols = ['No.', 'source', 'title_short', 'year', 'doi']
    print("\n후보 논문 목록:")
    print("=" * 80)
    for _, row in candidates_df[display_cols].iterrows():
        print(f"{row['No.']:2}. [{row['source']:6}] {row['title_short']}")
        print(f"    연도: {row['year']} | DOI: {row['doi']}")
        print()
else:
    print("검색 결과가 없습니다. 쿼리를 변경해보세요.")

---
## 2. 시드 논문 선택

**두 가지 방법 중 선택:**

1. **번호로 선택**: 위 검색 결과에서 번호 입력
2. **DOI 직접 입력**: 이미 알고 있는 논문의 DOI 입력

DOI가 입력되면 번호는 무시됩니다.

In [ ]:
# Cell 8: 시드 논문 선택 설정

# ============================================
# 방법 1: 위 검색 결과에서 번호로 선택
SEED_INDEX = 1  # <- 1부터 시작하는 번호

# 방법 2: DOI 직접 입력 (이 값이 있으면 SEED_INDEX 무시)
SEED_DOI = None  # <- DOI 입력 시 번호 무시
# 예시: SEED_DOI = "10.1038/s41591-023-02325-4"
# ============================================

print("시드 선택 설정 완료")
if SEED_DOI:
    print(f"  -> DOI 직접 입력: {SEED_DOI}")
else:
    print(f"  -> 검색 결과 {SEED_INDEX}번 선택")

In [ ]:
# Cell 9: Semantic Scholar API Helper

class SemanticScholarAPI:
    """Semantic Scholar API 래퍼"""
    
    BASE_URL = "https://api.semanticscholar.org/graph/v1"
    
    def __init__(self, api_key: str = None):
        self.api_key = api_key
        self.headers = {}
        if api_key:
            self.headers['x-api-key'] = api_key
    
    def _request(self, endpoint: str, params: dict = None) -> dict:
        """API 요청 (rate limit 처리 포함)"""
        url = f"{self.BASE_URL}{endpoint}"
        
        for attempt in range(3):
            try:
                response = requests.get(url, params=params, headers=self.headers)
                
                if response.status_code == 200:
                    return response.json()
                elif response.status_code == 429:
                    wait_time = 5 * (attempt + 1)
                    print(f"  Rate limit - {wait_time}초 대기...")
                    time.sleep(wait_time)
                elif response.status_code == 404:
                    # 논문을 찾을 수 없음 (정상적인 경우)
                    return {}
                else:
                    print(f"  API 오류: {response.status_code}")
                    return {}
            except Exception as e:
                print(f"  요청 오류: {e}")
                return {}
        
        return {}
    
    def get_paper_by_doi(self, doi: str) -> Dict:
        """DOI로 논문 정보 가져오기"""
        params = {
            "fields": "paperId,title,abstract,year,citationCount,authors"
        }
        result = self._request(f"/paper/DOI:{doi}", params)
        return result if result else {}
    
    def get_paper_by_title(self, title: str) -> Dict:
        """제목으로 논문 검색 (첫 번째 결과 반환)"""
        params = {
            "query": title,
            "limit": 1,
            "fields": "paperId,title,abstract,year,citationCount,authors"
        }
        result = self._request("/paper/search", params)
        if not result:
            return {}
        data = result.get('data', [])
        return data[0] if data else {}
    
    def get_references(self, paper_id: str, limit: int = 50) -> List[Dict]:
        """논문이 인용한 참조문헌 (References)"""
        params = {
            "fields": "paperId,title,abstract,year,citationCount",
            "limit": limit
        }
        result = self._request(f"/paper/{paper_id}/references", params)
        
        if not result:
            return []
        
        papers = []
        for item in result.get('data', []):
            cited = item.get('citedPaper')
            if cited and cited.get('paperId'):
                papers.append(cited)
        return papers
    
    def get_citations(self, paper_id: str, limit: int = 50) -> List[Dict]:
        """논문을 인용한 논문들 (Citations)"""
        params = {
            "fields": "paperId,title,abstract,year,citationCount",
            "limit": limit
        }
        result = self._request(f"/paper/{paper_id}/citations", params)
        
        if not result:
            return []
        
        papers = []
        for item in result.get('data', []):
            citing = item.get('citingPaper')
            if citing and citing.get('paperId'):
                papers.append(citing)
        return papers

# API 인스턴스 생성
api = SemanticScholarAPI(api_key=SEMANTIC_SCHOLAR_API_KEY)
print("✓ SemanticScholarAPI 준비 완료")

In [ ]:
# Cell 10: 시드 논문 가져오기

seed_paper = None

if SEED_DOI:
    # DOI로 직접 가져오기
    print(f"DOI로 논문 검색: {SEED_DOI}")
    seed_paper = api.get_paper_by_doi(SEED_DOI)
else:
    # 검색 결과에서 선택
    if len(candidates_df) >= SEED_INDEX:
        selected_row = candidates_df.iloc[SEED_INDEX - 1]
        
        # DOI가 있으면 DOI로, 없으면 제목으로 검색
        if selected_row.get('doi'):
            print(f"DOI로 논문 검색: {selected_row['doi']}")
            seed_paper = api.get_paper_by_doi(selected_row['doi'])
        else:
            print(f"제목으로 논문 검색: {selected_row['title'][:50]}...")
            seed_paper = api.get_paper_by_title(selected_row['title'])
    else:
        print(f"오류: {SEED_INDEX}번 논문이 없습니다. 1-{len(candidates_df)} 사이 값을 입력하세요.")

# 결과 확인
if seed_paper and seed_paper.get('paperId'):
    print("\n" + "=" * 60)
    print("선택된 시드 논문")
    print("=" * 60)
    print(f"제목: {seed_paper.get('title', 'N/A')}")
    print(f"연도: {seed_paper.get('year', 'N/A')}")
    print(f"인용수: {seed_paper.get('citationCount', 0)}")
    print(f"Paper ID: {seed_paper.get('paperId')}")
    
    # 저자 정보
    authors = seed_paper.get('authors', [])
    if authors:
        author_names = [a.get('name', '') for a in authors[:3]]
        print(f"저자: {', '.join(author_names)}" + (" 외" if len(authors) > 3 else ""))
else:
    print("\n시드 논문을 찾을 수 없습니다.")
    print("DOI를 확인하거나 다른 논문을 선택하세요.")

---
## 3. Citation Crawler 실행

시드 논문에서 시작해서:
1. **References**: 이 논문이 인용한 논문들
2. **Citations**: 이 논문을 인용한 논문들

```
         [인용한 논문들]      [인용된 논문들]
              ↑                    ↓
              └──── 시드 논문 ────┘
```

In [ ]:
# Cell 11: Citation Crawler 클래스

class CitationCrawler:
    """
    인용 네트워크를 탐색하는 Crawler
    """
    
    def __init__(self, api: SemanticScholarAPI):
        self.api = api
        self.visited: Set[str] = set()
        self.papers: Dict[str, Dict] = {}
    
    def crawl(
        self,
        seed_paper: Dict,
        max_hops: int = 1,
        refs_per_paper: int = 20,
        cites_per_paper: int = 20,
        max_total: int = 100
    ) -> List[Dict]:
        """
        시드 논문에서 인용 네트워크 탐색
        """
        self.visited.clear()
        self.papers.clear()
        
        # 시드 논문 추가
        paper_id = seed_paper.get('paperId')
        if not paper_id:
            print("오류: 시드 논문에 paperId가 없습니다.")
            return []
        
        self.visited.add(paper_id)
        seed_paper['hop'] = 0
        seed_paper['source'] = 'seed'
        self.papers[paper_id] = seed_paper
        
        current_level = [paper_id]
        print(f"시드 논문: {seed_paper.get('title', 'N/A')[:50]}...")
        
        # Multi-hop 탐색
        for hop in range(1, max_hops + 1):
            if len(self.papers) >= max_total:
                print(f"최대 논문 수 도달: {len(self.papers)}개")
                break
            
            print(f"\nHop {hop}: {len(current_level)}개 논문에서 확장 중...")
            next_level = []
            
            for pid in current_level:
                if len(self.papers) >= max_total:
                    break
                
                # References 가져오기
                refs = self.api.get_references(pid, limit=refs_per_paper)
                for ref in refs:
                    ref_id = ref.get('paperId')
                    if ref_id and ref_id not in self.visited:
                        self.visited.add(ref_id)
                        ref['hop'] = hop
                        ref['source'] = 'reference'
                        self.papers[ref_id] = ref
                        next_level.append(ref_id)
                
                # Citations 가져오기
                cites = self.api.get_citations(pid, limit=cites_per_paper)
                for cite in cites:
                    cite_id = cite.get('paperId')
                    if cite_id and cite_id not in self.visited:
                        self.visited.add(cite_id)
                        cite['hop'] = hop
                        cite['source'] = 'citation'
                        self.papers[cite_id] = cite
                        next_level.append(cite_id)
                
                time.sleep(0.3)  # Rate limit 방지
            
            print(f"  -> References: {len([p for p in self.papers.values() if p.get('source') == 'reference'])}개")
            print(f"  -> Citations: {len([p for p in self.papers.values() if p.get('source') == 'citation'])}개")
            print(f"  -> 현재 총: {len(self.papers)}개")
            current_level = next_level[:50]
        
        return list(self.papers.values())

print("CitationCrawler 클래스 정의 완료")

In [ ]:
# Cell 12: Crawler 실행
if seed_paper and seed_paper.get('paperId'):
    crawler = CitationCrawler(api)
    
    print("Citation Crawling 시작...")
    print("(API rate limit 때문에 시간이 걸릴 수 있습니다)\n")
    
    crawled_papers = crawler.crawl(
        seed_paper=seed_paper,
        max_hops=1,           # 1-hop (직접 인용만)
        refs_per_paper=15,    # 논문당 references 15개
        cites_per_paper=15,   # 논문당 citations 15개
        max_total=50          # 최대 50개
    )
    
    print(f"\n크롤링 완료! 총 {len(crawled_papers)}개 논문 수집")
else:
    print("시드 논문이 없습니다. Cell 10을 먼저 실행하세요.")
    crawled_papers = []

In [ ]:
# Cell 13: 크롤링 결과 통계
if crawled_papers:
    source_counts = Counter(p.get('source', 'unknown') for p in crawled_papers)
    years = [p.get('year') for p in crawled_papers if p.get('year')]
    
    print("=" * 50)
    print("크롤링 결과 통계")
    print("=" * 50)
    
    print("\n출처별 논문 수:")
    for source, count in source_counts.items():
        emoji = {"seed": "🌱", "reference": "📚", "citation": "📝"}.get(source, "📄")
        print(f"  {emoji} {source}: {count}개")
    
    if years:
        print(f"\n연도 범위: {min(years)} - {max(years)}")
        print(f"평균 연도: {sum(years) / len(years):.0f}")

---
## 4. SPECTER2 Selector로 필터링

Crawler가 수집한 논문 중에서 **쿼리와 가장 관련 있는 논문**만 선택합니다.

In [ ]:
# Cell 14: SPECTER2 Selector 정의
import torch
from torch.nn.functional import cosine_similarity

class SPECTER2Selector:
    """
    SPECTER2를 사용한 논문 관련성 평가
    """
    
    def __init__(self, device='cpu'):
        print("SPECTER2 모델 로딩 중...")
        print("(처음 실행 시 다운로드 ~500MB)")
        
        from transformers import AutoTokenizer
        from adapters import AutoAdapterModel
        
        self.tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')
        self.model = AutoAdapterModel.from_pretrained('allenai/specter2_base')
        self.model.load_adapter(
            "allenai/specter2", 
            source="hf", 
            load_as="proximity", 
            set_active=True
        )
        
        self.device = device
        self.model.to(device)
        self.model.eval()
        
        print(f"SPECTER2 로드 완료 (device: {device})")
    
    def get_embedding(self, title: str, abstract: str = "") -> torch.Tensor:
        """논문 임베딩 계산"""
        text = title + self.tokenizer.sep_token + (abstract or "")
        
        inputs = self.tokenizer(
            [text],
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=512
        ).to(self.device)
        
        with torch.no_grad():
            output = self.model(**inputs)
        
        return output.last_hidden_state[:, 0, :]
    
    def select_top_papers(
        self, 
        query: str, 
        papers: List[Dict], 
        top_k: int = 10
    ) -> List[Dict]:
        """
        쿼리와 관련성 높은 상위 K개 논문 선택
        """
        if not papers:
            return []
        
        print(f"\n{len(papers)}개 논문 평가 중...")
        
        query_emb = self.get_embedding(query)
        
        for i, paper in enumerate(papers):
            if (i + 1) % 10 == 0:
                print(f"  {i + 1}/{len(papers)} 완료")
            
            paper_emb = self.get_embedding(
                paper.get('title', ''),
                paper.get('abstract', '')
            )
            score = cosine_similarity(query_emb, paper_emb).item()
            paper['relevance_score'] = round(score, 4)
        
        papers.sort(key=lambda x: x.get('relevance_score', 0), reverse=True)
        
        print(f"평가 완료! 상위 {top_k}개 선택")
        return papers[:top_k]

print("SPECTER2Selector 클래스 정의 완료")

In [ ]:
# Cell 15: Selector 실행
if crawled_papers:
    selector = SPECTER2Selector(device='cpu')
    
    selected_papers = selector.select_top_papers(
        query=QUERY,
        papers=crawled_papers.copy(),
        top_k=10
    )
else:
    print("크롤링된 논문이 없습니다.")
    selected_papers = []

In [ ]:
# Cell 16: 최종 결과 출력
if selected_papers:
    print("=" * 70)
    print(f"SPECTER2 Selector 결과: Top 10")
    print(f"시드: {seed_paper.get('title', 'N/A')[:40]}...")
    print(f"쿼리: {QUERY}")
    print("=" * 70)
    
    for i, paper in enumerate(selected_papers, 1):
        title = paper.get('title', 'N/A')
        year = paper.get('year', 'N/A')
        score = paper.get('relevance_score', 0)
        source = paper.get('source', 'unknown')
        cites = paper.get('citationCount', 0)
        
        emoji = {"seed": "🌱", "reference": "📚", "citation": "📝"}.get(source, "📄")
        
        print(f"\n{i}. [{score:.3f}] {emoji} {title}")
        print(f"   연도: {year} | 인용: {cites} | 출처: {source}")

---
## 5. 시드 비교 실험

**다른 시드 논문을 선택하면 결과가 어떻게 달라질까요?**

전체 파이프라인을 함수로 묶어서, 시드만 바꿔가며 결과를 비교합니다.

In [ ]:
# Cell 17: 파이프라인 함수 정의

def run_pipeline(
    api: SemanticScholarAPI,
    selector: SPECTER2Selector,
    seed_doi: str = None,
    seed_title: str = None,
    query: str = "depression detection",
    max_papers: int = 30,
    top_k: int = 10
) -> Dict:
    """
    전체 파이프라인 실행
    
    Parameters:
    - seed_doi: 시드 논문 DOI (우선)
    - seed_title: 시드 논문 제목 (DOI 없을 때)
    - query: 관련성 평가용 쿼리
    
    Returns:
    - Dict with seed_paper, crawled_papers, selected_papers
    """
    result = {'seed': None, 'crawled': [], 'selected': []}
    
    # 1. 시드 논문 가져오기
    if seed_doi:
        seed = api.get_paper_by_doi(seed_doi)
    elif seed_title:
        seed = api.get_paper_by_title(seed_title)
    else:
        print("오류: seed_doi 또는 seed_title을 제공하세요.")
        return result
    
    if not seed or not seed.get('paperId'):
        print("시드 논문을 찾을 수 없습니다.")
        return result
    
    result['seed'] = seed
    print(f"시드: {seed.get('title', 'N/A')[:50]}...")
    
    # 2. Crawler 실행
    crawler = CitationCrawler(api)
    crawled = crawler.crawl(
        seed_paper=seed,
        max_hops=1,
        refs_per_paper=10,
        cites_per_paper=10,
        max_total=max_papers
    )
    result['crawled'] = crawled
    
    # 3. Selector 실행
    selected = selector.select_top_papers(
        query=query,
        papers=crawled.copy(),
        top_k=top_k
    )
    result['selected'] = selected
    
    return result

print("run_pipeline() 정의 완료")

In [ ]:
# Cell 18: 두 시드 비교 실험

# ============================================
# 비교할 두 시드 설정
# DOI 또는 제목 중 하나만 입력
# ============================================

# 시드 A: (예: 유명한 초기 논문)
SEED_A_DOI = None  # DOI 입력 또는
SEED_A_TITLE = "depression detection social media"  # 제목 입력

# 시드 B: (예: 최신 논문)
SEED_B_DOI = None
SEED_B_TITLE = "large language model mental health"

COMPARE_QUERY = "depression detection natural language processing"
# ============================================

print("=" * 60)
print("시드 A 파이프라인 실행")
print("=" * 60)
result_A = run_pipeline(
    api=api,
    selector=selector,
    seed_doi=SEED_A_DOI,
    seed_title=SEED_A_TITLE,
    query=COMPARE_QUERY,
    max_papers=30,
    top_k=10
)

print("\n" + "=" * 60)
print("시드 B 파이프라인 실행")
print("=" * 60)
result_B = run_pipeline(
    api=api,
    selector=selector,
    seed_doi=SEED_B_DOI,
    seed_title=SEED_B_TITLE,
    query=COMPARE_QUERY,
    max_papers=30,
    top_k=10
)

In [ ]:
# Cell 19: 결과 비교 분석

if result_A['selected'] and result_B['selected']:
    print("=" * 70)
    print("시드 비교 결과")
    print("=" * 70)
    
    # 시드 정보
    print(f"\n시드 A: {result_A['seed'].get('title', 'N/A')[:50]}...")
    print(f"        연도: {result_A['seed'].get('year')}, 인용: {result_A['seed'].get('citationCount', 0)}")
    print(f"\n시드 B: {result_B['seed'].get('title', 'N/A')[:50]}...")
    print(f"        연도: {result_B['seed'].get('year')}, 인용: {result_B['seed'].get('citationCount', 0)}")
    
    # 통계 비교
    print("\n" + "-" * 70)
    print(f"{'항목':<20} {'시드 A':>15} {'시드 B':>15}")
    print("-" * 70)
    print(f"{'크롤링 논문 수':<20} {len(result_A['crawled']):>15} {len(result_B['crawled']):>15}")
    
    # 평균 연도
    years_A = [p.get('year') for p in result_A['crawled'] if p.get('year')]
    years_B = [p.get('year') for p in result_B['crawled'] if p.get('year')]
    avg_year_A = sum(years_A) / len(years_A) if years_A else 0
    avg_year_B = sum(years_B) / len(years_B) if years_B else 0
    print(f"{'평균 연도':<20} {avg_year_A:>15.0f} {avg_year_B:>15.0f}")
    
    # Top 10 공통 논문
    ids_A = set(p.get('paperId') for p in result_A['selected'])
    ids_B = set(p.get('paperId') for p in result_B['selected'])
    common = ids_A & ids_B
    print(f"{'Top 10 공통 논문':<20} {len(common):>15} {'개':>14}")
    print("-" * 70)
    
    # 공통 논문 목록
    if common:
        print("\n공통으로 선택된 논문:")
        for p in result_A['selected']:
            if p.get('paperId') in common:
                print(f"  - {p.get('title', 'N/A')[:60]}...")
    else:
        print("\n공통 논문 없음 - 시드 선택이 결과에 큰 영향을 미침!")
else:
    print("비교할 결과가 없습니다.")

In [ ]:
# Cell 20: 시드별 Top 5 나란히 보기

if result_A['selected'] and result_B['selected']:
    print("=" * 90)
    print("시드별 Top 5 비교")
    print("=" * 90)
    
    print(f"\n{'순위':<4} {'시드 A Top 5':<40} {'시드 B Top 5':<40}")
    print("-" * 90)
    
    for i in range(5):
        title_A = result_A['selected'][i].get('title', 'N/A')[:35] + "..." if i < len(result_A['selected']) else "-"
        title_B = result_B['selected'][i].get('title', 'N/A')[:35] + "..." if i < len(result_B['selected']) else "-"
        print(f"{i+1:<4} {title_A:<40} {title_B:<40}")

---
## 6. DIY: 본인 논문으로 실험

본인이 잘 아는 논문의 DOI를 입력해서 실험해보세요.

In [ ]:
# DIY Cell: 본인 논문으로 실험

# ============================================
# 본인이 잘 아는 논문의 DOI 입력
MY_DOI = "10.xxxx/your-paper-doi"  # <- 수정!
# 또는 제목으로 검색
MY_TITLE = None  # <- DOI 대신 제목 입력 가능

MY_QUERY = "your research topic"  # <- 연구 주제 입력
# ============================================

print("파이프라인 실행 중...\n")

my_result = run_pipeline(
    api=api,
    selector=selector,
    seed_doi=MY_DOI if MY_DOI and not MY_DOI.startswith("10.xxxx") else None,
    seed_title=MY_TITLE,
    query=MY_QUERY,
    max_papers=30,
    top_k=10
)

if my_result['selected']:
    print("\n" + "=" * 60)
    print("내 시드 논문 기반 Top 10 추천")
    print("=" * 60)
    for i, p in enumerate(my_result['selected'], 1):
        score = p.get('relevance_score', 0)
        print(f"{i}. [{score:.3f}] {p.get('title', 'N/A')[:55]}...")

---
## 7. 결론

### 이번 노트북에서 배운 것

1. **paperscraper**: 다중 DB (arXiv, PubMed, bioRxiv) 통합 검색
2. **시드 선택**: 번호 또는 DOI로 시작점 지정
3. **Citation Crawler**: References + Citations 네트워크 확장
4. **SPECTER2 Selector**: 관련성 기반 필터링
5. **시드 비교**: 시드 선택이 결과에 미치는 영향 확인

### 핵심 인사이트

- **시드 선택이 중요**: 어떤 논문에서 시작하느냐에 따라 완전히 다른 논문들이 추천됨
- **오래된 시드 vs 최신 시드**: 인용 패턴이 다름
- **Crawler + Selector 조합**: 양(Crawler)과 질(Selector)의 균형

### 다음 단계
- **Part 3**: LitLLM으로 Related Work 초안 생성
- **Part 4**: AgentReview로 논문 피드백 받기

### 참고 자료
- [paperscraper GitHub](https://github.com/jannisborn/paperscraper)
- [Semantic Scholar API](https://api.semanticscholar.org/api-docs/)
- [SPECTER2 GitHub](https://github.com/allenai/SPECTER2)